## Train MM / explore with random sampling


In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools

# Number of combinations you want
num_combinations = 100  # Change this to however many combinations you need

# default
room_size = "xl-different-prob"
capacity_max = 24
batch_size = 4
terminates_at = 9
num_iterations = (terminates_at + 1) * 10
validation_starts_at = num_iterations // 2

prob_type = (
    "non-equal-object-probs" if "different-prob" in room_size else "equal-object-probs"
)
root_path = (
    f"./training-results/{prob_type}/dqn/room_size={room_size}/capacity={capacity_max}/"
)

root_path = "training-results/TRASH"

# random
test_seed_ = [i for i in range(num_combinations)]
target_update_interval_ = [10]
min_epsilon_ = [0.1]
gamma_ = [0.9, 0.99]
semantic_decay_factor_ = [0.7, 0.9, 0.99]
pretrain_semantic_ = [False, "include_walls", "exclude_walls"]

# Weights for agent_capacity_ elements
replay_buffer_size_ = [
    num_iterations,
    num_iterations // 2,
]
warm_start_ = [
    num_iterations // 2,
    num_iterations // 4,
    num_iterations // 10,
]


# Generate all combinations
params_all = list(
    itertools.product(
        test_seed_,
        target_update_interval_,
        min_epsilon_,
        gamma_,
        semantic_decay_factor_,
        pretrain_semantic_,
        replay_buffer_size_,
        warm_start_,
    )
)

# Random combinations with weighted agent_capacity_
random_combinations = random.sample(params_all, num_combinations)

for i, params in tqdm(enumerate(random_combinations)):
    (
        test_seed,
        target_update_interval,
        min_epsilon,
        gamma,
        semantic_decay_factor,
        pretrain_semantic,
        replay_buffer_size,
        warm_start,
    ) = params

    params_dict = {
        "env_str": "room_env:RoomEnv-v2",
        "num_iterations": num_iterations,
        "replay_buffer_size": replay_buffer_size,
        "validation_starts_at": validation_starts_at,
        "warm_start": warm_start,
        "batch_size": batch_size,
        "target_update_interval": target_update_interval,
        "epsilon_decay_until": num_iterations,
        "max_epsilon": 1.0,
        "min_epsilon": min_epsilon,
        "gamma": gamma,
        "capacity": {"long": capacity_max, "short": 15},
        "pretrain_semantic": pretrain_semantic,
        "semantic_decay_factor": semantic_decay_factor,
        "dqn_params": {
            "embedding_dim": 8,
            "num_layers_GNN": 2,
            "num_hidden_layers_MLP": 1,
            "dueling_dqn": True,
        },
        "num_samples_for_results": {"val": 5, "test": 10},
        "validation_interval": 5,
        "plotting_interval": 50,
        "train_seed": test_seed + 5,
        "test_seed": test_seed,
        "device": "cpu",
        "qa_function": "latest_strongest",
        "env_config": {
            "question_prob": 1.0,
            "terminates_at": terminates_at,
            "randomize_observations": "all",
            "room_size": room_size,
            "rewards": {"correct": 1, "wrong": 0, "partial": 0},
            "make_everything_static": False,
            "num_total_questions": 1000,
            "question_interval": 1,
            "include_walls_in_observations": True,
        },
        "ddqn": True,
        "default_root_dir": root_path,
    }

    agent = DQNAgent(**params_dict)
    agent.train()

## Run fixed combinations

In [ ]:
import matplotlib

matplotlib.use("Agg")

import logging

logger = logging.getLogger()
logger.disabled = True

import os
from agent import DQNAgent
from tqdm.auto import tqdm
import random
import itertools


room_size = "xl-different-prob"
terminates_at = 99
num_iterations = (terminates_at + 1) * 100
replay_buffer_size = num_iterations // 2
validation_starts_at = num_iterations // 2
warm_start = num_iterations // 4
batch_size = 32
target_update_interval = 10
gamma = 0.9
semantic_decay_factor = 0.9

for capacity_max in [24, 12, 6, 48]:
    prob_type = (
        "non-equal-object-probs"
        if "different-prob" in room_size
        else "equal-object-probs"
    )
    root_path = (
        f"./training-results/{prob_type}/dqn/"
        f"room_size={room_size}/capacity={capacity_max}/"
    )
    for pretrain_semantic in [False, "include_walls", "exclude_walls"]:
        for test_seed in [0, 1, 2, 3, 4]:
            params_dict = {
                "env_str": "room_env:RoomEnv-v2",
                "num_iterations": num_iterations,
                "replay_buffer_size": replay_buffer_size,
                "validation_starts_at": validation_starts_at,
                "warm_start": warm_start,
                "batch_size": batch_size,
                "target_update_interval": target_update_interval,
                "epsilon_decay_until": num_iterations,
                "max_epsilon": 1.0,
                "min_epsilon": 0.1,
                "gamma": gamma,
                "capacity": {"long": capacity_max, "short": 15},
                "pretrain_semantic": pretrain_semantic,
                "semantic_decay_factor": semantic_decay_factor,
                "dqn_params": {
                    "embedding_dim": 8,
                    "num_layers_GNN": 2,
                    "num_hidden_layers_MLP": 1,
                    "dueling_dqn": True,
                },
                "num_samples_for_results": {"val": 5, "test": 10},
                "validation_interval": 5,
                "plotting_interval": 50,
                "train_seed": test_seed + 5,
                "test_seed": test_seed,
                "device": "cpu",
                "qa_function": "latest_strongest",
                "env_config": {
                    "question_prob": 1.0,
                    "terminates_at": terminates_at,
                    "randomize_observations": "all",
                    "room_size": room_size,
                    "rewards": {"correct": 1, "wrong": 0, "partial": 0},
                    "make_everything_static": False,
                    "num_total_questions": 1000,
                    "question_interval": 1,
                    "include_walls_in_observations": True,
                },
                "ddqn": True,
                "default_root_dir": root_path,
            }

            agent = DQNAgent(**params_dict)
            agent.train()